In [51]:
import glob
import json
import os
import shutil
import operator
import sys
import argparse
import math
import copy

import numpy as np

MINOVERLAP = 0.5 

In [5]:
# args.ignore =[]
specific_iou_flagged = False
img_path = 'images'
show_animation = False
draw_plot = False


In [6]:
def log_average_miss_rate(precision, fp_cumsum, num_images):
    """
        log-average miss rate:
            Calculated by averaging miss rates at 9 evenly spaced FPPI points
            between 10e-2 and 10e0, in log-space.

        output:
                lamr | log-average miss rate
                mr | miss rate
                fppi | false positives per image

        references:
            [1] Dollar, Piotr, et al. "Pedestrian Detection: An Evaluation of the
               State of the Art." Pattern Analysis and Machine Intelligence, IEEE
               Transactions on 34.4 (2012): 743 - 761.
    """

    # if there were no detections of that class
    if precision.size == 0:
        lamr = 0
        mr = 1
        fppi = 0
        return lamr, mr, fppi

    fppi = fp_cumsum / float(num_images)
    mr = (1 - precision)

    fppi_tmp = np.insert(fppi, 0, -1.0)
    mr_tmp = np.insert(mr, 0, 1.0)

    # Use 9 evenly spaced reference points in log-space
    ref = np.logspace(-2.0, 0.0, num = 9)
    for i, ref_i in enumerate(ref):
        # np.where() will always find at least 1 index, since min(ref) = 0.01 and min(fppi_tmp) = -1.0
        j = np.where(fppi_tmp <= ref_i)[-1][-1]
        ref[i] = mr_tmp[j]

    # log(0) is undefined, so we use the np.maximum(1e-10, ref)
    lamr = math.exp(np.mean(np.log(np.maximum(1e-10, ref))))

    return lamr, mr, fppi

In [7]:
"""
 throw error and exit
"""
def error(msg):
    print(msg)
    sys.exit(0)

In [8]:

"""
 check if the number is a float between 0.0 and 1.0
"""
def is_float_between_0_and_1(value):
    try:
        val = float(value)
        if val > 0.0 and val < 1.0:
            return True
        else:
            return False
    except ValueError:
        return False

In [9]:
"""
 Calculate the AP given the recall and precision array
    1st) We compute a version of the measured precision/recall curve with
         precision monotonically decreasing
    2nd) We compute the AP as the area under this curve by numerical integration.
"""
def voc_ap(rec, prec):
    """
    --- Official matlab code VOC2012---
    mrec=[0 ; rec ; 1];
    mpre=[0 ; prec ; 0];
    for i=numel(mpre)-1:-1:1
            mpre(i)=max(mpre(i),mpre(i+1));
    end
    i=find(mrec(2:end)~=mrec(1:end-1))+1;
    ap=sum((mrec(i)-mrec(i-1)).*mpre(i));
    """
    rec.insert(0, 0.0) # insert 0.0 at begining of list
    rec.append(1.0) # insert 1.0 at end of list
    mrec = rec[:]
    prec.insert(0, 0.0) # insert 0.0 at begining of list
    prec.append(0.0) # insert 0.0 at end of list
    mpre = prec[:]
    """
     This part makes the precision monotonically decreasing
        (goes from the end to the beginning)
        matlab: for i=numel(mpre)-1:-1:1
                    mpre(i)=max(mpre(i),mpre(i+1));
    """
    # matlab indexes start in 1 but python in 0, so I have to do:
    #     range(start=(len(mpre) - 2), end=0, step=-1)
    # also the python function range excludes the end, resulting in:
    #     range(start=(len(mpre) - 2), end=-1, step=-1)
    for i in range(len(mpre)-2, -1, -1):
        mpre[i] = max(mpre[i], mpre[i+1])
    """
     This part creates a list of indexes where the recall changes
        matlab: i=find(mrec(2:end)~=mrec(1:end-1))+1;
    """
    i_list = []
    for i in range(1, len(mrec)):
        if mrec[i] != mrec[i-1]:
            i_list.append(i) # if it was matlab would be i + 1
    """
     The Average Precision (AP) is the area under the curve
        (numerical integration)
        matlab: ap=sum((mrec(i)-mrec(i-1)).*mpre(i));
    """
    ap = 0.0
    for i in i_list:
        ap += ((mrec[i]-mrec[i-1])*mpre[i])
    return ap, mrec, mpre

In [10]:
"""
 Convert the lines of a file to a list
"""
def file_lines_to_list(path):
    # open txt file lines to a list
    with open(path) as f:
        content = f.readlines()
    # remove whitespace characters like `\n` at the end of each line
    content = [x.strip() for x in content]
    return content

In [11]:
"""
 Draws text in image
"""
def draw_text_in_image(img, text, pos, color, line_width):
    font = cv2.FONT_HERSHEY_PLAIN
    fontScale = 1
    lineType = 1
    bottomLeftCornerOfText = pos
    cv2.putText(img, text,
            bottomLeftCornerOfText,
            font,
            fontScale,
            color,
            lineType)
    text_width, _ = cv2.getTextSize(text, font, fontScale, lineType)[0]
    return img, (line_width + text_width)

In [12]:
"""
 Plot - adjust axes
"""
def adjust_axes(r, t, fig, axes):
    # get text width for re-scaling
    bb = t.get_window_extent(renderer=r)
    text_width_inches = bb.width / fig.dpi
    # get axis width in inches
    current_fig_width = fig.get_figwidth()
    new_fig_width = current_fig_width + text_width_inches
    propotion = new_fig_width / current_fig_width
    # get axis limit
    x_lim = axes.get_xlim()
    axes.set_xlim([x_lim[0], x_lim[1]*propotion])

In [13]:
"""
 Draw plot using Matplotlib
"""
def draw_plot_func(dictionary, n_classes, window_title, plot_title, x_label, output_path, to_show, plot_color, true_p_bar):
    pass

In [14]:
"""
 这一块就是创建一些文件夹的，用来保存结果
 Create a ".temp_files/" and "results/" directory
"""
TEMP_FILES_PATH = ".temp_files"
if not os.path.exists(TEMP_FILES_PATH): # if it doesn't exist already
    os.makedirs(TEMP_FILES_PATH)
results_files_path = "results"
if os.path.exists(results_files_path): # if it exist already
    # reset the results directory
    shutil.rmtree(results_files_path)

os.makedirs(results_files_path)
if draw_plot:
    os.makedirs(results_files_path + "/classes")
if show_animation:
    os.makedirs(results_files_path + "/images")
    os.makedirs(results_files_path + "/images/single_predictions")


# 拿到ground_truth

有三个任务：

    1、按照图片保存ground_truth信息。
    2、保存每个类别出现过几个ground_truth
    3、保存每个类别出现在图片中的次数（一个图片中出现多个类别A也记为1）

In [63]:
"""
 Ground-Truth
     Load each of the ground-truth files into a temporary ".json" file.
     Create a list of all the class names present in the ground-truth (gt_classes).
"""
# get a list with the ground-truth files
# 格式是['ground-truth\\2007_000027.txt', 'ground-truth\\2007_000032.txt',...]这样的
ground_truth_files_list = glob.glob('ground-truth/*.txt')
if len(ground_truth_files_list) == 0:
    error("Error: No ground-truth files found!")
ground_truth_files_list.sort()

In [64]:
# dictionary with counter per class
gt_counter_per_class = {}
counter_images_per_class = {}

'''
    这里明确一下：
        1、一个*.txt文件对应一张*.jpg的图。
        2、一个*.txt文件中，会包含多个gt
'''
for txt_file in ground_truth_files_list:
    '''
        从ground-truth\\2007_000027.txt
        拿到了：2007_000027
    '''
    file_id = txt_file.split(".txt",1)[0]
    file_id = os.path.basename(os.path.normpath(file_id))
    # 看看对应的图片是否存在
    # check if there is a correspondent predicted objects file
    if not os.path.exists('predicted/' + file_id + ".txt"):
        error_msg = "Error. File not found: predicted/" + file_id + ".txt\n"
        error_msg += "(You can avoid this error message by running extra/intersect-gt-and-pred.py)"
        error(error_msg)

    # 拿到文件中的每行
    lines_list = file_lines_to_list(txt_file)
    # create ground-truth dictionary
    bounding_boxes = []
    is_difficult = False
    already_seen_classes = []
    '''
        1、对于这个图中的每个gt，拿到对应的类别和位置
        2、统计每个类别有几个gt。
        3、统计每个图片有几种gt。
    '''
    for line in lines_list:
        try:
            if "difficult" in line:
                    class_name, left, top, right, bottom, _difficult = line.split()
                    is_difficult = True
            else:
                    class_name, left, top, right, bottom = line.split()
        except ValueError:
            error_msg = "Error: File " + txt_file + " in the wrong format.\n"
            error_msg += " Expected: <class_name> <left> <top> <right> <bottom> ['difficult']\n"
            error_msg += " Received: " + line
            error_msg += "\n\nIf you have a <class_name> with spaces between words you should remove them\n"
            error_msg += "by running the script \"remove_space.py\" or \"rename_class.py\" in the \"extra/\" folder."
            error(error_msg)
        # check if class is in the ignore list, if yes skip
        if class_name in []:
            continue
        bbox = left + " " + top + " " + right + " " +bottom
        if is_difficult:
                bounding_boxes.append({"class_name":class_name, "bbox":bbox, "used":False, "difficult":True})
                is_difficult = False
        else:
                bounding_boxes.append({"class_name":class_name, "bbox":bbox, "used":False})

                '''
                记录每个种类出现过几次
                '''
                # count that object
                if class_name in gt_counter_per_class:
                    gt_counter_per_class[class_name] += 1
                else:
                    # if class didn't exist yet
                    gt_counter_per_class[class_name] = 1
                '''
                    这个感觉和上面的一样啊…
                '''
                if class_name not in already_seen_classes:
                    if class_name in counter_images_per_class:
                        counter_images_per_class[class_name] += 1
                    else:
                        # if class didn't exist yet
                        counter_images_per_class[class_name] = 1
                    already_seen_classes.append(class_name)


    # dump bounding_boxes into a ".json" file
    with open(TEMP_FILES_PATH + "/" + file_id + "_ground_truth.json", 'w') as outfile:
        json.dump(bounding_boxes, outfile)

In [65]:
bounding_boxes
'''
下面是每一个被存储起来的gt的样子，存储方式：
    1、按照图片被存储，每张图片一个文件。
    2、保存内容有gt的位置，类别，是否被使用
'''

'\n下面是每一个被存储起来的gt的样子，存储方式：\n    1、按照图片被存储，每张图片一个文件。\n    2、保存内容有gt的位置，类别，是否被使用\n'

In [66]:
'''
记录的是每个类别总共有几个gt
'''
print(gt_counter_per_class)

'''
每张图class去重之后，再记录
'''
print(counter_images_per_class)

{'pictureframe': 24, 'heater': 13, 'pottedplant': 29, 'book': 33, 'coffeetable': 22, 'tvmonitor': 20, 'bookcase': 7, 'doll': 8, 'vase': 12, 'shelf': 6, 'windowblind': 17, 'door': 29, 'wastecontainer': 11, 'nightstand': 7, 'sofa': 21, 'pillow': 45, 'chair': 106, 'diningtable': 47, 'remote': 8, 'cup': 36, 'cabinetry': 52, 'tap': 18, 'tincan': 28, 'countertop': 21, 'sink': 14, 'person': 7, 'bed': 8, 'bottle': 11, 'bowl': 15, 'backpack': 11}
{'pictureframe': 19, 'heater': 13, 'pottedplant': 22, 'book': 7, 'coffeetable': 16, 'tvmonitor': 16, 'bookcase': 7, 'doll': 8, 'vase': 10, 'shelf': 6, 'windowblind': 15, 'door': 28, 'wastecontainer': 11, 'nightstand': 5, 'sofa': 21, 'pillow': 21, 'chair': 46, 'diningtable': 38, 'remote': 8, 'cup': 21, 'cabinetry': 45, 'tap': 18, 'tincan': 18, 'countertop': 21, 'sink': 14, 'person': 6, 'bed': 8, 'bottle': 10, 'bowl': 12, 'backpack': 7}


In [67]:
'''
    拿到所有出现过的class类别
    并排序，取长度啥的
'''
gt_classes = list(gt_counter_per_class.keys())
# let's sort the classes alphabetically
gt_classes = sorted(gt_classes)
n_classes = len(gt_classes)

# 拿到prediction信息

任务：

    1、按照不同的类别，保存各个预测测bbox。

In [68]:
# 读取预测的信息
# get a list with the predicted files
predicted_files_list = glob.glob('predicted/*.txt')
predicted_files_list.sort()

In [69]:
'''
    这里很有意思，不是读取每一个predict文件，而是按照类别来
    1、预测文件按照类别进行保存
'''

# 对于每一个类别
for class_index, class_name in enumerate(gt_classes):
    bounding_boxes = []
    # 遍历每一个预测文件
    for txt_file in predicted_files_list:
        #print(txt_file)
        # the first time it checks if all the corresponding ground-truth files exist
        file_id = txt_file.split(".txt",1)[0]
        file_id = os.path.basename(os.path.normpath(file_id))
        # 第一次进来检查一下
        if class_index == 0:
            if not os.path.exists('ground-truth/' + file_id + ".txt"):
                error_msg = "Error. File not found: ground-truth/" +    file_id + ".txt\n"
                error_msg += "(You can avoid this error message by running extra/intersect-gt-and-pred.py)"
                error(error_msg)

        lines = file_lines_to_list(txt_file)
        # 对于文件中的每一行
        # 这些行应该是已经做过NMS的
        for line in lines:
            # 拿到分类,置信度和坐标信息
            try:
                tmp_class_name, confidence, left, top, right, bottom = line.split()
            except ValueError:
                error_msg = "Error: File " + txt_file + " in the wrong format.\n"
                error_msg += " Expected: <class_name> <confidence> <left> <top> <right> <bottom>\n"
                error_msg += " Received: " + line
                error(error_msg)
            # 如果是当前类别的bbox，那么保存
            if tmp_class_name == class_name:
                #print("match")
                bbox = left + " " + top + " " + right + " " +bottom
                bounding_boxes.append({"confidence":confidence, "file_id":file_id, "bbox":bbox})
                #print(bounding_boxes)

    # 按照置信度降序保存
    # sort predictions by decreasing confidence
    bounding_boxes.sort(key=lambda x:float(x['confidence']), reverse=True)
    with open(TEMP_FILES_PATH + "/" + class_name + "_predictions.json", 'w') as outfile:
        json.dump(bounding_boxes, outfile)

In [75]:
sum_AP = 0.0
ap_dictionary = {}
lamr_dictionary = {}
# open file to store the results
with open(results_files_path + "/results.txt", 'w') as results_file:
    results_file.write("# AP and precision/recall per class\n")
    count_true_positives = {}
    # 对于每个类别
    for class_index, class_name in enumerate(gt_classes):
        count_true_positives[class_name] = 0
        """
         Load predictions of that class
         拿到这个类别的预测结果：
            1、这个预测结果都是属于这个类别的。
            2、这个预测结果是跨图片的。
        """
        predictions_file = TEMP_FILES_PATH + "/" + class_name + "_predictions.json"
        predictions_data = json.load(open(predictions_file)) 

        """
         Assign predictions to ground truth objects
        """
        nd = len(predictions_data)
        tp = [0] * nd # creates an array of zeros of size nd
        fp = [0] * nd
              
        '''
        对于这个类的每一个预测结果：
            1、拿到对应图片的名字
            2、拿到这个图片对应的gt_file
                
        '''
        for idx, prediction in enumerate(predictions_data):
            # 拿到对应图片的名字
            file_id = prediction["file_id"]
           
            # assign prediction to ground truth object if any
            # open ground-truth with that file_id
            # 拿到这个图片对应的gt_file
            gt_file = TEMP_FILES_PATH + "/" + file_id + "_ground_truth.json"
            ground_truth_data = json.load(open(gt_file))
            ovmax = -1
            gt_match = -1
            # load prediction bounding-box
            # 拿到这个bbox的坐标
            bb = [ float(x) for x in prediction["bbox"].split() ]
            # 对于这个gt_file中的每个gt
            temp_gtd = copy.deepcopy(ground_truth_data)
            for obj in temp_gtd:
                # look for a class_name match
                # 如果类别相同，计算IOU
                if obj["class_name"] == class_name:
                    bbgt = [ float(x) for x in obj["bbox"].split() ]
                    bi = [max(bb[0],bbgt[0]),
                          max(bb[1],bbgt[1]),
                          min(bb[2],bbgt[2]),
                          min(bb[3],bbgt[3])]
                    iw = bi[2] - bi[0] + 1
                    ih = bi[3] - bi[1] + 1
                    # 计算IOU
                    if iw > 0 and ih > 0:
                        # compute overlap (IoU) = area of intersection / area of union
                        ua = (bb[2] - bb[0] + 1) * (bb[3] - bb[1] + 1) + (bbgt[2] - bbgt[0]
                                        + 1) * (bbgt[3] - bbgt[1] + 1) - iw * ih
                        ov = iw * ih / ua
                        # 计算这个bbox和哪个类别相同的gt的IOU最高。
                        if ov > ovmax:
                            ovmax = ov
                            gt_match = obj

            # assign prediction as true positive/don't care/false positive
           
            # set minimum overlap
            min_overlap = MINOVERLAP # 0.5
           
            '''
            来，看看什么时候才算一个TP:
            1、对于某一个类别A；
            2、拿到这个类别A的所有预测框。
            3、对于某一个预测框b；
            4、拿到它对应的图片p；
            5、拿到这个图片上所有类别为A的gts。
            6、计算b和gts中哪个gt的IOU最大。
            7、如果IOU超过了设定的阈值，那么算一个tp
            8、如果IOU没超过，算一个FP
            9、如果IOU超过了，但是这个gt被用过了，算一个FP
            '''
            if ovmax >= min_overlap:
                if "difficult" not in gt_match:
                        # 这个gt还没有用过
                        if not bool(gt_match["used"]):
                            '''
                                1、这里的idx表示的是这个类别的第几个预测框
                                2、因此，这里会有len(gt_class)个tp，每个tp的长度都不一样。
                                3、fp同理。
                            '''
                            # true positive
                            tp[idx] = 1
                            gt_match["used"] = True
                            count_true_positives[class_name] += 1
                            # update the ".json" file
                            with open(gt_file, 'w') as f:
                                    f.write(json.dumps(ground_truth_data))
                            if show_animation:
                                status = "MATCH!"
                        else:
                            # false positive (multiple detection)
                            fp[idx] = 1
                            if show_animation:
                                status = "REPEATED MATCH!"
            else:
                # false positive
                fp[idx] = 1
                if ovmax > 0:
                    status = "INSUFFICIENT OVERLAP"

        print(class_name,predictions_data,len(predictions_data))
        print(tp,fp)
                    
        '''
            其实这里是这样：
                1、观察predictions_data，即同一个类别的预测框。它是按照属于这个类别的置信度进行降序排列的。
                2、那么这里的相加，表示，当我的阈值不断放松的时候，对于那些已经度过了IOU关卡的预测bbox而言，会有多少bbox能够被收容。
                3、比如过了属于backpack类别的预测bbox只有5个，且他们过了IOU的tp是[1,0,1,1,0],表示出了第1个和第4个与gt的iou太小被删除，
                   其他都在。且从0到4，他们是属于backpack类别的置信度是降序排列的。
                4、然后从头累加，得到[1,1,2,3,3],表示当阈值设置的最大，只有1个bbox是tp；当阈值设置第次大，也只有1个bbox是tp；
                   当阈值再小，就有2个bbox是tp了…以此类推。
                5、fp的操作也一样。
                6、这样，就完成了这个类别的precision-recall曲线
        '''
        cumsum = 0
        for idx, val in enumerate(fp):
            fp[idx] += cumsum
            cumsum += val
        cumsum = 0
        for idx, val in enumerate(tp):
            tp[idx] += cumsum
            cumsum += val
            
        
        print(class_name,gt_counter_per_class[class_name])
        #print(tp)
        rec = tp[:]
        for idx, val in enumerate(tp):
            # 是对每一个图？
            rec[idx] = float(tp[idx]) / gt_counter_per_class[class_name]
        #print(rec)
        prec = tp[:]
        for idx, val in enumerate(tp):
            # 是对每一个图？
            prec[idx] = float(tp[idx]) / (fp[idx] + tp[idx])
        #print(prec)
        
        print(tp,fp)

        print('recal:',rec)
        print('prec',prec)


        '''
            1、这个是算面积的函数
            利用的是precision monotonically decreasing方法。
            2、具体而言，对于recall为r时刻的precision，我们将其设置为：
                2.1 取得所有recall为r',满足r'>r时刻的precision。
                2.2 将recall为r时刻的precision，设置为2.1中的最大值。
            3、实际操作上，对于precision：pre = [1.0, 0.5, 0.6666666666666666, 0.75, 0.6]
               我们认为precision逐渐变小，是由于阈值逐渐变小
               而阈值逐渐变小，表示recall逐渐变大。
            4、所以我们默认pre这个list后面的对应的recall值，大于前面的。
            5、那么我们就只要从后往前，累计求极大就好了。
            
        '''
        ap, mrec, mprec = voc_ap(rec[:], prec[:])
        
        if class_index==1:
            break
        
        sum_AP += ap
        text = "{0:.2f}%".format(ap*100) + " = " + class_name + " AP " #class_name + " AP = {0:.2f}%".format(ap*100)
        """
         Write to results.txt
        """
        rounded_prec = [ '%.2f' % elem for elem in prec ]
        rounded_rec = [ '%.2f' % elem for elem in rec ]
        results_file.write(text + "\n Precision: " + str(rounded_prec) + "\n Recall :" + str(rounded_rec) + "\n\n")
        print(text)
        ap_dictionary[class_name] = ap

        n_images = counter_images_per_class[class_name]
        lamr, mr, fppi = log_average_miss_rate(np.array(rec), np.array(fp), n_images)
        lamr_dictionary[class_name] = lamr

       

  

    results_file.write("\n# mAP of all classes\n")
    mAP = sum_AP / n_classes
    text = "mAP = {0:.2f}%".format(mAP*100)
    results_file.write(text + "\n")
    print(text)

backpack [{'confidence': '0.552314', 'file_id': '2007_000837', 'bbox': '45 216 220 313'}, {'confidence': '0.552256', 'file_id': '2007_000648', 'bbox': '126 249 332 455'}, {'confidence': '0.411606', 'file_id': '2007_000762', 'bbox': '374 231 568 466'}, {'confidence': '0.374395', 'file_id': '2007_000661', 'bbox': '287 230 466 470'}, {'confidence': '0.350580', 'file_id': '2007_000822', 'bbox': '20 133 121 333'}] 5
[1, 0, 1, 1, 0] [0, 1, 0, 0, 1]
backpack 11
[1, 1, 2, 3, 3] [0, 1, 1, 1, 2]
recal: [0.09090909090909091, 0.09090909090909091, 0.18181818181818182, 0.2727272727272727, 0.2727272727272727]
prec [1.0, 0.5, 0.6666666666666666, 0.75, 0.6]
22.73% = backpack AP 
bed [{'confidence': '0.936491', 'file_id': '2007_000452', 'bbox': '1 88 599 473'}, {'confidence': '0.930039', 'file_id': '2007_000837', 'bbox': '12 91 586 467'}, {'confidence': '0.870608', 'file_id': '2007_000515', 'bbox': '0 92 442 448'}, {'confidence': '0.848061', 'file_id': '2007_000799', 'bbox': '5 96 478 456'}, {'confidenc

In [86]:
def voc_ap(rec, prec):
    
    rec.insert(0, 0.0) # insert 0.0 at begining of list
    rec.append(1.0) # insert 1.0 at end of list
    mrec = rec[:]
    
    prec.insert(0, 0.0) # insert 0.0 at begining of list
    prec.append(0.0) # insert 0.0 at end of list
    mpre = prec[:]
    print(mpre)
    for i in range(len(mpre)-2, -1, -1):
        print(mpre[i], mpre[i+1])
        mpre[i] = max(mpre[i], mpre[i+1])
    print(mpre)

    i_list = []
    for i in range(1, len(mrec)):
        if mrec[i] != mrec[i-1]:
            i_list.append(i) # if it was matlab would be i + 1
    print(mrec)
    print(i_list)
    ap = 0.0
    for i in i_list:
        ap += ((mrec[i]-mrec[i-1])*mpre[i])
    return ap, mrec, mpre

In [87]:
rec = [0.09090909090909091, 0.09090909090909091, 0.18181818181818182, 0.2727272727272727, 0.2727272727272727]
pre = [1.0, 0.5, 0.6666666666666666, 0.75, 0.6]
voc_ap(rec,pre)

[0.0, 1.0, 0.5, 0.6666666666666666, 0.75, 0.6, 0.0]
0.6 0.0
0.75 0.6
0.6666666666666666 0.75
0.5 0.75
1.0 0.75
0.0 1.0
[1.0, 1.0, 0.75, 0.75, 0.75, 0.6, 0.0]
[0.0, 0.09090909090909091, 0.09090909090909091, 0.18181818181818182, 0.2727272727272727, 0.2727272727272727, 1.0]
[1, 3, 4, 6]


(0.22727272727272724,
 [0.0,
  0.09090909090909091,
  0.09090909090909091,
  0.18181818181818182,
  0.2727272727272727,
  0.2727272727272727,
  1.0],
 [1.0, 1.0, 0.75, 0.75, 0.75, 0.6, 0.0])